In [1]:
import pandas as pd
# from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_excel('Online Retail.xlsx')

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
import seaborn as sns 
import matplotlib.pyplot as plt

In [4]:
df.dropna(inplace=True)

In [5]:
sub = df.sample(n=10000, random_state=1)

In [6]:
sub.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
443133,574691,22341,LOVE GARLAND PAINTED ZINC,3,2011-11-06 13:17:00,1.65,14174.0,United Kingdom
415129,572517,22562,MONSTERS STENCIL CRAFT,2,2011-10-24 14:47:00,1.25,15068.0,United Kingdom


In [7]:
thing = sub.groupby(['CustomerID', 'StockCode'], as_index=False).count()

In [8]:
grouped_index = df.groupby(['CustomerID', 'StockCode'], as_index=False).size()

In [9]:
grouped_index['StockCode'] = grouped_index['StockCode'].astype(str)

In [10]:
grouped_index['StockCode'] += ' '

In [11]:
final = grouped_index.groupby(['CustomerID'], as_index=False)['StockCode'].sum()

In [12]:
final.head(2)

,CustomerID,StockCode
0,12346.0,23166
1,12347.0,16008 17021 20665 20719 20780 20782 20966 2103...


In [13]:
def tokenize(text):
    tokens = text.split()
    return tokens

In [14]:
final['StockCode'] = final['StockCode'].apply(lambda x: tokenize(x))

In [15]:
final

,CustomerID,StockCode
0,12346.0,[23166]
1,12347.0,"[16008, 17021, 20665, 20719, 20780, 20782, 209..."
2,12348.0,"[21211, 21213, 21725, 21726, 21967, 21977, 219..."
3,12349.0,"[20685, 20914, 21086, 21136, 21231, 21232, 214..."
4,12350.0,"[20615, 20652, 21171, 21832, 21864, 21866, 219..."
...,...,...
4367,18280.0,"[22084, 22180, 22358, 22467, 22495, 22499, 226..."
4368,18281.0,"[22028, 22037, 22467, 22716, 23007, 23008, 23209]"
4369,18282.0,"[21108, 21109, 21270, 22089, 22423, 22424, 226..."
4370,18283.0,"[20676, 20712, 20717, 20718, 20719, 20723, 207..."


Build Word2Vec

In [18]:
# # Defining the window for context
# window = 2

# # Creating a placeholder for the scanning of the word list
# word_lists = []
# all_text = []

# for text in sentences:
    
#     # Cleaning the text
#     text = clean_text(text)
#     # Appending to the all text list
#     all_text += text 

#     # Creating a context dictionary
#     for i, word in enumerate(text):
#         for w in range(window):
#             # Getting the context that is ahead by *window* words
#             if i + 1 + w < len(text): 
#                 word_lists.append([word] + [text[(i + 1 + w)]])
#             # Getting the context that is behind by *window* words    
#             if i - w - 1 >= 0: 
#                 word_lists.append([word] + [text[(i - w - 1)]])

In [26]:
words_list = []

def count_words(list_row):
    for word in list_row:
        if word not in words_list:
            words_list.append(word)
    

In [27]:
final.head()

,CustomerID,StockCode
0,12346.0,[23166]
1,12347.0,"[16008, 17021, 20665, 20719, 20780, 20782, 209..."
2,12348.0,"[21211, 21213, 21725, 21726, 21967, 21977, 219..."
3,12349.0,"[20685, 20914, 21086, 21136, 21231, 21232, 214..."
4,12350.0,"[20615, 20652, 21171, 21832, 21864, 21866, 219..."


In [28]:
final['StockCode'].apply(lambda x: count_words(x))

0       None
1       None
2       None
3       None
4       None
        ... 
4367    None
4368    None
4369    None
4370    None
4371    None
Name: StockCode, Length: 4372, dtype: object

In [31]:
len(words_list)

3685

In [33]:
def create_unique_word_dict(text:list) -> dict:
    """
    A method that creates a dictionary where the keys are unique words
    and key values are indices
    """
    # Getting all the unique words from our text and sorting them alphabetically
    words = list(set(text))
    words.sort()

    # Creating the dictionary for the unique words
    unique_word_dict = {}
    for i, word in enumerate(words):
        unique_word_dict.update({
            word: i
        })

    return unique_word_dict 

In [37]:
unique_word_dict = create_unique_word_dict(words_list)

In [51]:
from scipy import sparse
import numpy as np
from tqdm import tqdm
# Defining the number of features (unique words)
n_words = len(unique_word_dict)

# Getting all the unique words 
words = list(unique_word_dict.keys())

# Creating the X and Y matrices using one hot encoding
X = []
Y = []

for i, word_list in tqdm(enumerate(words_list)):
    
    # Getting the indices
    main_word_index = unique_word_dict.get(words_list[0])
    context_word_index = unique_word_dict.get(words_list[1])

    # Creating the placeholders   
    X_row = np.zeros(n_words)
    Y_row = np.zeros(n_words)

    # One hot encoding the main word
    X_row[main_word_index] = 1

    # One hot encoding the Y matrix words 
    Y_row[context_word_index] = 1

    # Appending to the main matrices
    X.append(X_row)
    Y.append(Y_row)

# Converting the matrices into an array
X = np.asarray(X)
Y = np.asarray(Y)

3685it [00:00, 15697.92it/s]


In [52]:
# Deep learning: 
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense

# Defining the size of the embedding
embed_size = 2

# Defining the neural network
inp = Input(shape=(X.shape[1],))
x = Dense(units=embed_size, activation='linear')(inp)
x = Dense(units=Y.shape[1], activation='softmax')(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

# Optimizing the network weights
model.fit(
    x=X, 
    y=Y, 
    batch_size=256,
    epochs=1000
    )

# Obtaining the weights from the neural network. 
# These are the so called word embeddings

# The input layer 
weights = model.get_weights()[0]

# Creating a dictionary to store the embeddings in. The key is a unique word and 
# the value is the numeric vector
embedding_dict = {}
for word in words: 
    embedding_dict.update({
        word: weights[unique_word_dict.get(word)]
        })

Epoch 1/1000
15/15 [==============================] - 2s 21ms/step - loss: 8.1968
Epoch 2/1000
15/15 [==============================] - 0s 20ms/step - loss: 8.1613
Epoch 3/1000
15/15 [==============================] - 0s 21ms/step - loss: 8.1201
Epoch 4/1000
15/15 [==============================] - 0s 20ms/step - loss: 8.0715
Epoch 5/1000
15/15 [==============================] - 0s 21ms/step - loss: 8.0144
Epoch 6/1000
15/15 [==============================] - 0s 20ms/step - loss: 7.9480
Epoch 7/1000
15/15 [==============================] - 0s 20ms/step - loss: 7.8720
Epoch 8/1000
15/15 [==============================] - 0s 20ms/step - loss: 7.7860
Epoch 9/1000
15/15 [==============================] - 0s 20ms/step - loss: 7.6900
Epoch 10/1000
15/15 [==============================] - 0s 21ms/step - loss: 7.5838
Epoch 11/1000
15/15 [==============================] - 0s 21ms/step - loss: 7.4675
Epoch 12/1000
15/15 [==============================] - 0s 20ms/step - loss: 7.3410
Epoch 13/1000

In [54]:
embedding_dict

{'10002': array([0.0043015 , 0.03577998], dtype=float32),
 '10080': array([-0.01597055, -0.00371064], dtype=float32),
 '10120': array([-0.03381294, -0.01984824], dtype=float32),
 '10123C': array([0.02254175, 0.03585073], dtype=float32),
 '10124A': array([-0.01011246, -0.02934796], dtype=float32),
 '10124G': array([-0.03433429,  0.00636997], dtype=float32),
 '10125': array([ 0.03785406, -0.04020194], dtype=float32),
 '10133': array([ 0.0400277 , -0.00540416], dtype=float32),
 '10135': array([0.02262783, 0.01360926], dtype=float32),
 '11001': array([0.00370392, 0.00097417], dtype=float32),
 '15030': array([0.02298935, 0.01066931], dtype=float32),
 '15034': array([ 0.0315669 , -0.00184745], dtype=float32),
 '15036': array([-0.00498356,  0.02745414], dtype=float32),
 '15039': array([ 0.01849879, -0.02690415], dtype=float32),
 '15044A': array([-0.03972035,  0.02971452], dtype=float32),
 '15044B': array([-0.00107546, -0.03587677], dtype=float32),
 '15044C': array([-0.03290327, -0.01850808], 